In [ ]:
import os
import sys
sys.path.append('..') 
import yaml
import pandas as pd
import numpy as np
import pickle
from src.utils.utils_p import YamlParser,Upload_Download_Pickle,Config_Paths,Feature_Selection_Configs,Model_Configs,Create_Labels_Configs
from src.data.CollectData.CollectDataFromCSV import *
from src.preprocessing.Data_Preprocessing import *
from src.combineDatasets.Combine_Datasets import *
from src.featureProcessing.SplitDataset.Split_Train_Test import *
from src.featureProcessing.SplitDataset.Vertical_Split import *
from src.featureProcessing.Feature_Selection.Basic_Elemination import *
from src.featureProcessing.Feature_Selection.Wrapper import *
from src.featureProcessing.Feature_Selection.Hybrid import *
from src.featureProcessing.Feature_Extraction.Normalize import *
#from src.processing.Model import *
from src.model.Classification.Random_Forest_Tuning import *
from src.model.Classification.XGBoost_Tuning import *
from src.model.Classification.LightGBM_Tuning import *
from src.model.Classification.CatBoost_Tuning import *
from src.model.Classification.SVC_Tuning import *
from src.model.Classification.Voting import *
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [ ]:
path=Config_Paths().get_processed_path()
cpath=Config_Paths().get_combineddatasets_path()
rpath=Config_Paths().get_raw_path()
prepath=Config_Paths().get_intermediate_path()


In [ ]:
#PERIOD yerine YEAR_MONTH_DAY geldi.
LogsT02=Upload_Download_Pickle().download_pickle(prepath, 'Features_not_normalT02')
LabelsT02=Upload_Download_Pickle().download_pickle(prepath, 'Labels_not_normalT02')
LogsT03=Upload_Download_Pickle().download_pickle(prepath, 'Features_not_normalT03')
LogsT52=Upload_Download_Pickle().download_pickle(prepath, 'Features_not_normalT52')


In [ ]:
LogsT02.State_Fault.astype(int).unique()

In [ ]:
LogsT02=pd.merge(LogsT02,LabelsT02[['Pitch systemEv_multiclass','Pitch systemEv','Period']])

In [ ]:
from datetime import datetime, timedelta
LogsT02['onedayPeriod']=LogsT02['Period'].astype('datetime64[ns]')- timedelta(days=1)
LogsT02['fivedaysPeriod']=LogsT02['Period'].astype('datetime64[ns]')- timedelta(days=5)
LogsT02['fiftheendaysPeriod']=LogsT02['Period'].astype('datetime64[ns]')- timedelta(days=15)


In [ ]:
LogsT02=LogsT02.set_index('Period')
tag=LogsT02[LogsT02['Pitch systemEv']=='EVENT_134']


In [ ]:
for i in tag.index:
    print(i)
    print(tag.loc[i,'onedayPeriod'])
    LogsT02['New_Pitch_systemEv']=np.where((LogsT02.index<=i)&(LogsT02.index>tag.loc[i,'fiftheendaysPeriod']),'EVENT_134_fifthteen',0)    
    LogsT02['New_Pitch_systemEv']=np.where((LogsT02.index<=i)&(LogsT02.index>tag.loc[i,'fivedaysPeriod']),'EVENT_134_five',LogsT02['New_Pitch_systemEv'])  
    LogsT02['New_Pitch_systemEv']=np.where((LogsT02.index<=i)&(LogsT02.index>tag.loc[i,'onedayPeriod']),'EVENT_134_one',LogsT02['New_Pitch_systemEv'])
  
    

In [ ]:
LogsT02 ['New_Pitch_systemEv'].unique()

In [ ]:
tag_plot=LogsT02[LogsT02['New_Pitch_systemEv']!=0]

In [ ]:
#CollectData from given dataset
CollectDataFromCSV().upload_all_datasets() 

In [ ]:
col_22=[col.replace(" 22 ", " ") for col in LogsT22.columns]
col_02=[col.replace(" 02 ", " ") for col in LogsT02.columns]

In [ ]:
[items for items in col_02 if items not in col_22]

In [ ]:
[items for items in col_22 if items not in col_02]

In [ ]:
LogsT22=Upload_Download_Pickle().download_pickle(rpath, 'LogsT22')

In [ ]:
FaultsT22=Upload_Download_Pickle().download_pickle(rpath, 'FaultsT22')

In [ ]:
LogsT22.describe()

In [ ]:
# Duration'ı saat'e çevir onu al!
FaultsT22['Duration (hh:mm:ss)'].dtypes

In [ ]:
# Hata ne sıklıkla geliyor bunu al!!
Fault_count=FaultsT22[['Event ID','Category','Description','Lost Production (kWh)']].groupby(['Event ID','Category','Description']).count()
# Hata toplam Lost:
Fault_total_Lost=FaultsT22[['Event ID','Category','Description','Lost Production (kWh)']].groupby(['Event ID','Category','Description']).sum().sort_values(by='Lost Production (kWh)',ascending=False)

## En son hepsini merge et!

In [ ]:
sum_list=LogsT02.isna().sum()

In [ ]:
#CollectData from given dataset
CollectDataFromCSV().upload_all_datasets() 

#Clearn-Normalize-Combine dataset
Data_Preprocessing().preprocessing()

In [ ]:
Features_not_normalT02=Upload_Download_Pickle().download_pickle(prepath, 'Features_not_normalT02')
Labels_not_normalT02=Upload_Download_Pickle().download_pickle(prepath, 'Labels_not_normalT02')
Features_not_normalT22=Upload_Download_Pickle().download_pickle(prepath, 'Features_not_normalT22')
Labels_not_normalT22=Upload_Download_Pickle().download_pickle(prepath, 'Labels_not_normalT22')
Logs_all_fillT22=Upload_Download_Pickle().download_pickle(prepath, 'Logs_all_fillT22')
Logs_all_fillT02=Upload_Download_Pickle().download_pickle(prepath, 'Logs_all_fillT02')

In [ ]:
Labels_not_normalT02["Events_binary"].unique()

In [ ]:
Labels_not_normalT02.columns

In [ ]:
Features_not_normalT02.head()

In [ ]:
#Combine Multiple Identical Datasets
Combine_Datasets().multi_combine_dataset()

In [ ]:
#PERIOD yerine YEAR_MONTH_DAY geldi.
Features_df=Upload_Download_Pickle().download_pickle(cpath, 'Features_df')
AllLabels_df=Upload_Download_Pickle().download_pickle(cpath, 'AllLabels_df')
Labels_df=Upload_Download_Pickle().download_pickle(cpath, 'Labels_df')


In [ ]:
Features_df[Features_df['TURBINE_NUMBER'].isin(['T22'])].head()

In [ ]:
Features_df.describe()

In [ ]:
Features_df[Features_df['Events_binary']==1]['State_Fault'].unique()

In [ ]:
Vertical_Split().split_train_testset()

In [ ]:
#Split Hold-0ut Dataset
Split_Train_Test().split_train_testset()

In [ ]:
X_test=Upload_Download_Pickle().download_pickle(path, 'X_test')
y_test=Upload_Download_Pickle().download_pickle(path, 'y_test')
X_trains=Upload_Download_Pickle().download_pickle(path, 'X_trains')
y_trains=Upload_Download_Pickle().download_pickle(path, 'y_trains')

In [ ]:
Normalize().featureExtraction()

In [ ]:
Basic_Elemination().featureSelection()

In [ ]:
meta=Upload_Download_Pickle().download_pickle(path, 'meta')

In [ ]:
meta.groupby('STATUS').count()

In [ ]:
#Feature Selection with Hybrid method
Hybrid().featureSelection()

In [ ]:
meta=Upload_Download_Pickle().download_pickle(path, 'meta')

In [ ]:
meta.groupby('STATUS').count()

In [ ]:
X_trains_ext=Upload_Download_Pickle().download_pickle(path, 'X_trains_ext')
X_trains_list_ext=Upload_Download_Pickle().download_pickle(path, 'X_trains_list_ext')
X_test_ext=Upload_Download_Pickle().download_pickle(path, 'X_test_ext')
X_test_list_ext=Upload_Download_Pickle().download_pickle(path, 'X_test_list_ext')

In [ ]:
X_test_list_ext[0].head()

In [ ]:
X_trains_ext.dtypes

In [ ]:
XGBoost_Tune().Model()

In [ ]:
# Create & Predict 
XGBoost_Tune().Model()
LightGBM_Tune().Model()
CatBoost_Tune().Model()
#Random_Forest_Tune().Model()
#Voting().Model()
#SVC_Tune().Model()

In [ ]:
Random_Forest_Tune().Model()

In [ ]:
processed_path=Config_Paths().get_processed_path()
selectedfeatures = Upload_Download_Pickle().download_pickle(processed_path, 'selectedfeatures')


In [ ]:
selectedfeatures

In [ ]:
# Create Model Metrics according to test sets
XGBoost_Tune().Model_Metrics('classification')

In [ ]:
# Create Model Metrics according to test sets
XGBoost_Tune().Model_Metrics('classification')
LightGBM_Tune().Model_Metrics('classification')
CatBoost_Tune().Model_Metrics('classification')
Random_Forest_Tune().Model_Metrics('classification')
#SVC_Tune().Model_Metrics('classification')

In [ ]:
# Plot Results of Grid Search
XGBoost_Tune().grid_plot()
LightGBM_Tune().grid_plot()
CatBoost_Tune().grid_plot()
Random_Forest_Tune().grid_plot()
SVC_Tune().grid_plot()

In [ ]:
# Plot Results of Grid Search
XGBoost_Tune().grid_plot()